## Práctica 7. Visualización de datos geoespaciales con Leaflet para R

Con esta práctica se busca reforzar los conceptos y las principales funciones de Leaflet para manipular y desplegar datos geo-espaciales interactivos en R.

Vamos a desplegar información sobre población y densidad de población por estado en la República Mexicana, con base en datos del Censo de Población y Vivienda 2020 de INEGI.

### 1. Carga de las librerías necesarias


In [ ]:
install.packages("sf")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘proxy’, ‘e1071’, ‘wk’, ‘classInt’, ‘Rcpp’, ‘s2’, ‘units’




### 2. Carga de los datos

Los archivos que vamos a utilizar  se encuentran en Canvas y ya han sido depurados y limpiados. Generalmente no tenemos esa suerte. Una de las tareas principales de un científico de datos, es identificar las fuentes de datos, validarlas y depurarlas.

El archivo `MapaMexico.geojson` contiene una cartografía con los polígonos de los estados del país.

El archivo `estadosMx.csv` contiene información sobre los estados de la República Mexicana.

Lea los archivos y asígnelos a variables apropiadas



In [ ]:
df1 <- read.csv("estadosMx.csv")

mxpolig <- st_read("MapaMexico.geojson")

---
**PREGUNTA 1** ¿Qué clase de objeto son df1 y mxpolig?
¿Qué campos contiene un registro en mxpolig?

---

### 3. Nuestro primer mapa

Ya tenemos lo necesario para desplegar nuestro primer mapa. Invocamos a leaflet pasando como argumento el mapa de los polígonos con el mapa base por omisión.  Para evitar que se despliegue todo el mapa mundi, especificamos las coordenadas y el acercamiento deseados con la función `setView()`.

In [ ]:
# 1. Veamos los estados con el mapa base
# setView para acercar el mapa a Mx
mp1 <- leaflet(mxpolig) %>%
  addTiles()%>%
  setView(lng=-99.2,lat=19.34,zoom=7)

mp1


In [ ]:
---
  **PREGUNTA 2** Muestre las imágenes que obtiene haciendo un acercamiento con los valores 2 y 7.

---

### 4. Añadir marcadores

Ahora vamos añadir marcadores a las capitales de los Estados; es información que se encuentra en nuestro otro dataset.
Como esa información contiene coordenadas geográficas, ya no es necesario invocar la función `setView()`, pero si lo prefiere, puede utilizarla.

In [ ]:
# 2. Añadimos marcadores. Como tienen coordenadas, podemos eliminar el setView

mp2 <- leaflet(mxpolig) %>%
  addTiles() %>%
  addMarkers(data=df1)%>%
  setView(lng=-99.2,lat=19.34,zoom=4.4)
mp2




### 5. Popups en los marcadores

Recordemos que podemos hacer que al dar clic en un marcador (o en otro objeto), leaflet pueda desplegar cierta información. Estos son los popups. Vamos a crear unos que desplieguen el nombre del Estado y su población:

In [ ]:
# 3. Añadimos mensajes popup a los marcadores
mp2 <- mp2 %>%
  addMarkers(data=df1,popup = paste0(
    "<strong>Nombre: </strong>", df1$NOM_ENT, "<br>",
    "<strong>Población: </strong>", df1$POB, "<br>"))
mp2


---
**PREGUNTA 3** ¿Para qué sirven las etiquetas `<strong>` y `<br>` en HTML?  
Añada un tercer renglón con la densidad poblacional del Estado.
De clic en algún estado para que se despliegue la información y haga una captura de pantalla.

---



### 6. Agrupamiento de marcadores

Leaflet tiene una muy útil función para agrupar marcadores que se encuentran muy juntos y desplegar en su lugar un círculo con el número de marcadores agrupados:



In [ ]:
# 4. Agregamos agrupamiento de marcadores

mp3 <- leaflet(mxpolig) %>%
  addTiles() %>%
  addMarkers(data=df1,popup = paste0(
    "<strong>Nombre: </strong>", df1$NOM_ENT, "<br>",
    "<strong>Poblaci?n: </strong>", df1$POB, "<br>"),
    clusterOptions = markerClusterOptions())
mp3




---

**PREGUNTA 4** Haga un zoom en el que se muestren desagregados algunos de los clusters pero no todos. Haga una captura de pantalla.

---



### 7. Opciones para Mapa Base

Como sabemos, con la función `addProviderTiles()` podemos elegir el mapa base sobre el que se desplegarán nuestras visualizaciones.  Lo interesante ahora, es que con la función `addLayerControls()` podemos seleccionar en el mapa interactivo qué mapa base deseamos desplegar



In [ ]:
# 5. Añadimos opciones para mapas base

mp4 <- leaflet(mxpolig) %>%
  addTiles(group="Mapa base 1") %>%
  addMarkers(data=df1,popup = paste0(
    "<strong>Nombre: </strong>", df1$NOM_ENT, "<br>",
    "<strong>Población: </strong>", df1$POB, "<br>"),
    clusterOptions = markerClusterOptions()) %>%
  addProviderTiles(providers$Stamen.Toner,group="Mapa base 2")%>%
  addProviderTiles(providers$Esri.NatGeoWorldMap,group="Mapa base 3")%>%
  addLayersControl(
    baseGroups = c("Mapa base 1", "Mapa base 2", "Mapa base 3"),
    options=layersControlOptions(collapsed=F)
)


mp4




---

**PREGUNTA 5** ¿Qué ocurre si declaramos `collapsed=TRUE` en el snippet anterior?

Muestre (de preferencia en un solo renglón) capturas de pantalla con cada uno de los mapas base

---



### 8. Mapas de coropletas

Leaflet tiene varias funciones que simplifican el mapeo de valores numéricos (o categóricos) a paletas de colores. Son las siguientes:

* **colorNumeric:**  Datos continuos, paleta de colores continua

* **colorBin**: Datos continuos, paleta de colores discreta

* **colorQuantile**: Datos continuos, paleta de colores discreta, mismo número de observaciones para cada tonalidad

* **colorFactor**: Datos categóricos

  

#### 8.1 Coropletas para población

Vamos a utilizar `colorNumeric()` para colorear cada estado de acuerdo al tamaño de su población.  Empezamos por crear la paleta con tonalidades de azul e indicamos en la función `addPolygons()` que deseamos que el color de cada polígono corresponda a la paleta de acuerdo al campo que tiene los datos de población.

Todos los demás argumentos son opciones con las que puede jugar.  Por ejemplo, con la opción `label` podemos hacer aparecer una etiqueta con la información que deseemos para cada polígono al pasar encima de él.

In [ ]:
# 5. Mapas de coropletas

# 5.1 Por No. habitantes con colorbin

# Paleta de colores - continuo continuo
palPob <- colorNumeric("Blues",domain=df1$POB)

mp5<-leaflet(mxpolig) %>%
  addTiles() %>%
  addMarkers(data=df1,popup = paste0(
    "<strong>Nombre: </strong>", df1$NOM_ENT, "<br>",
    "<strong>Población: </strong>", df1$POB, "<br>"),
    clusterOptions = markerClusterOptions()) %>%
  addPolygons(stroke = F,
              smoothFactor = 0.2,
              opacity=1.0,
              fillOpacity = 0.5,
              fillColor = ~palPob(df1$POB),
              highlightOptions = highlightOptions(color="white",
                                                  weight=2,
                                                  bringToFront = T),
              label=df1$NOM_ENT,
              labelOptions = labelOptions(direction = "auto"))%>%
  addLegend(position="bottomleft", pal=palPob, values = ~df1$POB,
            title="Población")


mp5




---

**PREGUNTA 6** Muestre una captura de pantalla donde se observe la distinta tonalidad del Edo. de México, CDMX, Morelos y Puebla

¿Cuál es la población de Jalisco y la de Colima? (Ayuda: Está en el Popup)

---



#### 8.2 Coropletas para densidad

Podemos utilizar un código muy parecido para hacer un mapa de coropletas para la densidad poblacional.
Vamos a probar con un número discreto de colores (6) en tonalidades de verde:

In [ ]:
palDens <- colorBin("Greens", domain = df1$DENS, 6, pretty = F)



El problema es que hay una enorme diversidad en las densidades poblacionales: De 10 a casi 6000 habitantes por kilómetro cuadrado.

Una forma de resolver esto, es indicarle a R cómo queremos separar los bins. Después de un breve análisis de los datos, encontramos los puntos de corte para mantener 6 niveles.  Los cortes y el código son:

In [ ]:
bins <- c(0, 20, 45, 79, 150, 390, 800, 7000)
palDens <- colorBin("Greens", domain = df1$DENS, bins=bins, pretty = F)

---
**PREGUNTA 8** Modifique su código y muestre el mapa de coropletas resultante

---



#### 8.3  Juntando los dos mapas de coropletas

Así como pudimos seleccionar a través de botones de radio (*radiobuttons*) qué mapa base utilizar, podemos elegir a través de botones de selección qué mapa de coropletas desplegar.  Suponiendo que al mapa de población se le asignó el grupo "Por población" y al de densidad el grupo "Por densidad", el código para activar el menú de selección es el siguiente:

In [ ]:
addLayersControl(overlayGroups = c("Por Población", "Por Densidad"))



---

**PREGUNTA 9** Integre los dos mapas en un solo programa.  Muestre una captura con los dos mapas activados, sólo con el de población y sólo con densidad

---



### 9 Juntándolo todo

---

**PREGUNTA 10**  Muy bien, ahora agregue el código para que también pueda elegir cualquiera de los tres mapas base que hizo en la primera parte.

Muestre las capturas de pantalla que considere convenientes para demostrar la funcionalidad de su código

---

